In [1]:
import json
from tqdm import tqdm, trange

The signs we used for some earlier model:

In [2]:
final = {0: 'Curve Left',
 1: 'Curve Right',
 2: 'Speed Bump',
 6: 'Pedestrian Crossing Soon',
 12: 'Left - Right Curve',
 13: 'Right - Left Curve',
 22: 'Roundabout Soon',
 28: '50m - Railroad',
 29: '100m - Railroad',
 30: '150m - Railroad',
 35: 'Bumpy Road',
 37: 'Yield',
 43: 'STOP',
 50: 'Forbidden For Everything',
 51: 'Speed Limit - 10',
 52: 'Speed Limit - 30',
 53: 'Speed Limit - 40',
 54: 'Speed Limit - 50',
 55: 'Speed Limit - 60',
 56: 'Speed Limit - 70',
 59: 'No parking/waiting',
 60: 'No stopping',
 62: 'No entry',
 64: 'Pedestrian Path',
 68: 'Proceed Straight',
 69: 'Turn Right',
 70: 'Turn Left',
 71: 'Keep straight or Turn Left',
 72: 'Keep Straight or Turn Right',
 73: 'Turn Left or Right',
 74: 'Pass onto Right',
 75: 'Pass Onto Left',
 76: 'Roundabout Here',
 103: 'End Of No - Overtake',
 104: 'End of 40 Limit',
 105: 'End of 50 Limit',
 106: 'End of 60 Limit',
 107: 'End of 70 Limit',
 112: 'End of All Restrictions',
 116: 'No Stopping Zone',
 117: '30km Limit Zone',
 118: '40km Limit Zone',
 119: 'Priority Road',
 120: 'End of 30km Limit Zone',
 123: 'Parking',
 132: 'Bicycle Crossing',
 135: 'Info',
 136: 'Pedestrian Crossing',
 157: 'Disabilities',
 158: 'Towing',
 159: 'Rain',
 160: 'Snow'}

Remap in increasing order:

In [3]:
mapping = dict()
start = 0
for key in final:
  mapping[key] = start
  start += 1

Finally obtain a mapping from class id to sign name:

In [4]:
final_mapping = {mapping[key]:value for key, value in final.items()}

In [5]:
final_mapping

{0: 'Curve Left',
 1: 'Curve Right',
 2: 'Speed Bump',
 3: 'Pedestrian Crossing Soon',
 4: 'Left - Right Curve',
 5: 'Right - Left Curve',
 6: 'Roundabout Soon',
 7: '50m - Railroad',
 8: '100m - Railroad',
 9: '150m - Railroad',
 10: 'Bumpy Road',
 11: 'Yield',
 12: 'STOP',
 13: 'Forbidden For Everything',
 14: 'Speed Limit - 10',
 15: 'Speed Limit - 30',
 16: 'Speed Limit - 40',
 17: 'Speed Limit - 50',
 18: 'Speed Limit - 60',
 19: 'Speed Limit - 70',
 20: 'No parking/waiting',
 21: 'No stopping',
 22: 'No entry',
 23: 'Pedestrian Path',
 24: 'Proceed Straight',
 25: 'Turn Right',
 26: 'Turn Left',
 27: 'Keep straight or Turn Left',
 28: 'Keep Straight or Turn Right',
 29: 'Turn Left or Right',
 30: 'Pass onto Right',
 31: 'Pass Onto Left',
 32: 'Roundabout Here',
 33: 'End Of No - Overtake',
 34: 'End of 40 Limit',
 35: 'End of 50 Limit',
 36: 'End of 60 Limit',
 37: 'End of 70 Limit',
 38: 'End of All Restrictions',
 39: 'No Stopping Zone',
 40: '30km Limit Zone',
 41: '40km Limit

Files with the original dataset annotations:

In [6]:
train_difficult_path = "DFG-tsd-aug-annot-json-with-difficult-samples/train-difficult.json"
test_difficult_path = "DFG-tsd-annot-json/test.json"

In [7]:
with open(train_difficult_path) as file:
  train_difficult_json = json.load(file)

In [8]:
with open(test_difficult_path) as file:
  test_json = json.load(file)

The annotation dict that we are building:

In [9]:
annotations_dict = {key['id']:{
  'name': key['file_name'],
  'height': key['height'],
  'width': key['width'],
  'annotations': []
} for key in sorted(train_difficult_json['images'] + test_json['images'], key = lambda image: image['id'])}

In [10]:
len(annotations_dict.keys())

15732

Build the annotations dict:

In [11]:
for annotation in tqdm(train_difficult_json['annotations']):
  image_id = annotation['image_id']
  category_id = annotation['category_id']
  if category_id not in final:
    continue
  im_width, im_height = annotations_dict[image_id]['width'], annotations_dict[image_id]['height']
  xmin, ymin, width, height = annotation["bbox"]
  xcenter, ycenter, width, height = xmin + width/2, ymin + height/2, width, height
  xcenter, ycenter, width, height = xcenter / im_width, ycenter / im_height, width / im_width, height / im_height
  annotations_dict[image_id]['annotations'].append({
    'xcenter': xcenter,
    'ycenter': ycenter,
    'width': width,
    'height': height,
    'category_id': mapping[category_id]
  })

100%|██████████| 44091/44091 [00:00<00:00, 658105.93it/s]


In [12]:
annotations_dict[0]

{'name': '0000001.jpg',
 'height': 1080,
 'width': 1920,
 'annotations': [{'xcenter': 0.18541666666666667,
   'ycenter': 0.30833333333333335,
   'width': 0.05416666666666667,
   'height': 0.07037037037037037,
   'category_id': 49},
  {'xcenter': 0.028645833333333332,
   'ycenter': 0.2851851851851852,
   'width': 0.026041666666666668,
   'height': 0.06296296296296296,
   'category_id': 47},
  {'xcenter': 0.18255208333333334,
   'ycenter': 0.15,
   'width': 0.05260416666666667,
   'height': 0.13333333333333333,
   'category_id': 20}]}

In [13]:
mapping

{0: 0,
 1: 1,
 2: 2,
 6: 3,
 12: 4,
 13: 5,
 22: 6,
 28: 7,
 29: 8,
 30: 9,
 35: 10,
 37: 11,
 43: 12,
 50: 13,
 51: 14,
 52: 15,
 53: 16,
 54: 17,
 55: 18,
 56: 19,
 59: 20,
 60: 21,
 62: 22,
 64: 23,
 68: 24,
 69: 25,
 70: 26,
 71: 27,
 72: 28,
 73: 29,
 74: 30,
 75: 31,
 76: 32,
 103: 33,
 104: 34,
 105: 35,
 106: 36,
 107: 37,
 112: 38,
 116: 39,
 117: 40,
 118: 41,
 119: 42,
 120: 43,
 123: 44,
 132: 45,
 135: 46,
 136: 47,
 157: 48,
 158: 49,
 159: 50,
 160: 51}

In [14]:
inverse_mapping = {value:key for key, value in mapping.items()}

Build annotation dict for test set:

In [15]:
for annotation in tqdm(test_json['annotations']):
  image_id = annotation['image_id']
  category_id = annotation['category_id']
  if category_id not in final:
    continue
  im_width, im_height = annotations_dict[image_id]['width'], annotations_dict[image_id]['height']
  xmin, ymin, width, height = annotation["bbox"]
  xcenter, ycenter, width, height = xmin + width/2, ymin + height/2, width, height
  xcenter, ycenter, width, height = xcenter / im_width, ycenter / im_height, width / im_width, height / im_height
  annotations_dict[image_id]['annotations'].append({
    'xcenter': xcenter,
    'ycenter': ycenter,
    'width': width,
    'height': height,
    'category_id': mapping[category_id]
  })

100%|██████████| 4393/4393 [00:00<00:00, 549033.89it/s]


In [16]:
annotations_dict[1]

{'name': '0000002.jpg',
 'height': 1080,
 'width': 1920,
 'annotations': [{'xcenter': 0.16302083333333334,
   'ycenter': 0.45416666666666666,
   'width': 0.0375,
   'height': 0.06759259259259259,
   'category_id': 30},
  {'xcenter': 0.7575520833333333,
   'ycenter': 0.21203703703703702,
   'width': 0.027604166666666666,
   'height': 0.03888888888888889,
   'category_id': 11}]}

Save a .txt file for each image with its annotations:

In [17]:
for image_index, image_data in annotations_dict.items():
  name = image_data['name'][:-4]
  yolo_format = ''
  if len(image_data['annotations']) == 0:
    continue
  for annotation in image_data['annotations']:
    xcenter, ycenter, width, height, category_id = annotation.values()
    yolo_format += f'{category_id} {xcenter} {ycenter} {width} {height}\n'
  with open(f'dataset/JPEGImages/labels/{name}.txt', 'w') as file:
    file.write(yolo_format)